In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path

pd.set_option('display.max_columns', 300)

In [2]:
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Nottingham Forest', 'Tottenham', 'Sunderland', 'West Ham', 'Wolves']

In [3]:
# fetch FPL data online
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])
fpl_online_df['team_name'] = [teams[i] for i in fpl_online_df['team']-1]
fpl_online_df['name'] = fpl_online_df.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_online_df['season'] = '25-26'
fpl_online_df

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,birth_date,has_temporary_code,opta_code,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,team_name,name,season
0,True,True,NaN,None,154561,0,0,0,0,0,1,4.5,None,0,David,0.0,1,False,,None,55,154561.jpg,3.7,False,Raya Martín,18.4,False,None,a,1,3,142,0,0,0,0,0.0,25.8,Raya,200.0,2024-07-04,1995-09-15,False,p154561,3420,0,0,13,34,0,0,0,3,0,86,10,555,755.4,10.7,0.0,76.7,0,38,0.00,0.03,0.03,35.03,37,8,342,11,630,74,178,8,NaN,,NaN,,NaN,,0.0,2.26,0.00,0.00,0.92,0.89,124,4,471,74,68,11,18,2,1.00,0.34,Arsenal,David Raya Martín,25-26
1,True,True,NaN,None,109745,0,0,0,0,0,1,2.8,None,0,Kepa,0.0,2,False,,None,45,109745.jpg,3.4,False,Arrizabalaga Revuelta,1.0,False,None,a,1,3,106,0,0,0,0,0.0,23.6,Arrizabalaga,200.0,2025-07-01,1994-10-03,False,p109745,2790,0,0,8,39,0,0,0,3,0,97,7,406,744.4,10.4,4.0,75.8,0,31,0.06,0.03,0.09,38.83,38,9,344,12,333,1,183,9,NaN,,NaN,,NaN,,0.0,3.13,0.00,0.00,1.25,1.26,363,21,142,49,94,16,192,28,1.00,0.26,Arsenal,Kepa Arrizabalaga Revuelta,25-26
2,True,True,NaN,None,463748,0,0,0,0,0,1,1.5,None,0,Karl,0.0,3,False,,None,40,463748.jpg,0.0,False,Hein,0.7,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Hein,67.0,2020-10-28,2002-04-13,False,p463748,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.00,0.00,435,53,425,44,394,29,435,53,NaN,,NaN,,NaN,,0.0,0.00,0.00,0.00,0.00,0.00,596,58,56,27,438,53,226,31,0.00,0.00,Arsenal,Karl Hein,25-26
3,True,True,NaN,None,551221,0,0,0,0,0,1,1.5,None,0,Tommy,0.0,4,False,,None,40,551221.jpg,0.0,False,Setford,0.2,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Setford,241.0,2024-07-24,2006-03-13,False,p551221,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.00,0.00,469,64,460,55,434,42,469,64,NaN,,NaN,,NaN,,0.0,0.00,0.00,0.00,0.00,0.00,639,71,99,40,472,64,441,59,0.00,0.00,Arsenal,Tommy Setford,25-26
4,True,True,NaN,None,226597,0,0,0,0,0,2,4.5,None,0,Gabriel,0.0,5,False,,None,60,226597.jpg,4.2,False,dos Santos Magalhães,26.9,False,None,a,1,3,117,0,0,0,0,0.0,19.5,Gabriel,30.0,2020-09-01,1997-12-19,False,p226597,2363,3,2,10,22,0,0,0,4,0,0,9,459,584.6,208.8,287.0,108.2,0,28,2.70,1.21,3.91,23.09,96,32,167,36,107,12,121,31,NaN,,NaN,,NaN,,0.1,0.00,0.05,0.15,0.88,0.84,95,5,573,217,32,1,11,4,1.07,0.38,Arsenal,Gabriel dos Santos Magalhães,25-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,True,True,NaN,None,44998

In [41]:
# Statshead data
data1 = pd.read_csv('../../../misc/pl24-25_data1.csv', header=[0,1])

# edit column names that have unnamed main headers
new_columns = [('General',col[1]) if 'Unnamed' in col[0] \
                else col for col in data1.columns]
data1.columns = pd.MultiIndex.from_tuples(new_columns)

# remove duplicate columns, keeping the last occurrence
data1 = data1.loc[:,~data1.columns.duplicated(keep='last')] 

data1

General                                                       \
         Rk                  Player        Date     Age   Nation   
0         1                 Murillo  2025-01-14  22-194   br BRA   
1         2            Dean Huijsen  2025-03-09  19-329   es ESP   
2         3                  Morato  2024-12-29  23-182   br BRA   
3         4         Maxence Lacroix  2025-02-22  24-322   fr FRA   
4         5            Jan Bednarek  2024-12-29  28-261   pl POL   
..      ...                     ...         ...     ...      ...   
795     796            Mikel Merino  2025-04-20  28-302   es ESP   
796     797            Sander Berge  2025-01-18  26-339   no NOR   
797     798  Pervis Estupi\'f1\'e1n  2024-11-09  26-293   ec ECU   
798     799             Diogo Dalot  2024-12-30  25-287   pt POR   
799     800             Cole Palmer  2025-05-04  22-363  eng ENG   

                                                                              \
                Team Unnamed: 8_level_1              Opp                Comp   
0    Nott'ham Forest                NaN        Liverpool  eng Premier League   
1        Bournemouth                  @        Tottenham  eng Premier League   
2    Nott'ham Forest                  @          Everton  eng Premier League   
3     Crystal Palace                  @           Fulham  eng Premier League   
4        Southampton                  @   Crystal Palace  eng Premier League   
..               ...                ...              ...                 ...   
795          Arsenal                  @     Ipswich Town  eng Premier League   
796           Fulham                  @   Leicester City  eng Premier League   
797         Brighton                NaN  Manchester City  eng Premier League   
798   Manchester Utd                NaN    Newcastle Utd  eng Premier League   
799          Chelsea                NaN        Liverpool  eng Premier League   

                     Performance                           Tackles       \
    Result Start Min         Gls Ast G+A G-PK PK PKatt PKm     Tkl TklW   
0    D 1-1     Y  90           0   0   0    0  0     0   0       4    4   
1    D 2-2     Y  90           0   0   0    0  0     0   0       2    1   
2    W 2-0     Y  90           0   0   0    0  0     0   0       4    3   
3    W 2-0     Y  90           0   0   0    0  0     0   0       4    3   
4    L 1-2     Y  90           0   0   0    0  0     0   0       4    2   
..     ...   ...  ..         ...  ..  ..  ... ..   ...  ..     ...  ...   
795  W 4-0     Y  56           0   1   1    0  0     0   0       6    4   
796  W 2-0     Y  90           0   0   0    0  0     0   0       6    3   
797  W 2-1     Y  90           0   0   0    0  0     0   0       4    2   
798  L 0-2     Y  90           0   0   0    0  0     0   0       5    2   
799  W 3-1     Y  90           1   0   1    0  1     1   0       2    1   

                            Challenges                 Blocks         General  \
    Def 3rd Mid 3rd Att 3rd        Tkl Att   Tkl% Lost Blocks Sh Pass     Int   
0         3       1       0          1   2   50.0    1      4  1    3       2   
1         2       0       0          2   3   66.7    1      1  1    0       3   
2         4       0       0          2   3   66.7    1      1  0    1       1   
3         3       1       0          2   2  100.0    0      2  1    1       1   
4         1       3       0          1   2   50.0    1      1  1    0       6   
..      ...     ...     ...        ...  ..    ...  ...    ... ..  ...     ...   
795       1       1       4          3   3  100.0    0      0  0    0       0   
796       2       4       0          3   3  100.0    0      1  0    1       1   
797       2       2       0          1   4   25.0    3      0  0    0       2   
798       2       3       0          2   3   66.7    1      1  0    1       1   
799       0       2       0          1   2   50.0    1      0  0    0       3   

                                        -additio

In [42]:
# Statshead data
data2 = pd.read_csv('../../../misc/pl24-25_data2.csv', header=[0,1])

# edit column names that have unnamed main headers
new_columns = [('General',col[1]) if 'Unnamed' in col[0] \
                else col for col in data2.columns]
data2.columns = pd.MultiIndex.from_tuples(new_columns)

# remove duplicate columns, keeping the last occurrence
data2 = data2.loc[:,~data2.columns.duplicated(keep='last')] 

data2

General                                                          \
         Rk                     Player        Date     Age   Nation   
0         1              Ryan Christie  2025-01-18  29-331  sct SCO   
1         2              Neco Williams  2025-05-18  24-035  wls WAL   
2         3               Jan Bednarek  2025-02-15  28-309   pl POL   
3         4  Victor Bernth Kristiansen  2025-02-27  22-073   dk DEN   
4         5              Wilfred Ndidi  2024-12-03  27-353   ng NGA   
..      ...                        ...         ...     ...      ...   
595     596             Marc Cucurella  2025-03-16  26-237   es ESP   
596     597             Marc Cucurella  2025-03-09  26-230   es ESP   
597     598             Marc Cucurella  2025-04-20  26-272   es ESP   
598     599              Jacob Greaves  2025-05-25  24-255  eng ENG   
599     600              Jacob Greaves  2025-05-18  24-248  eng ENG   

                                                                             \
                Team Unnamed: 8_level_1             Opp                Comp   
0        Bournemouth                  @   Newcastle Utd  eng Premier League   
1    Nott'ham Forest                  @        West Ham  eng Premier League   
2        Southampton                NaN     Bournemouth  eng Premier League   
3     Leicester City                  @        West Ham  eng Premier League   
4     Leicester City                NaN        West Ham  eng Premier League   
..               ...                ...             ...                 ...   
595          Chelsea                  @         Arsenal  eng Premier League   
596          Chelsea                NaN  Leicester City  eng Premier League   
597          Chelsea                  @          Fulham  eng Premier League   
598     Ipswich Town                NaN        West Ham  eng Premier League   
599     Ipswich Town                  @  Leicester City  eng Premier League   

                     Performance                           Tackles       \
    Result Start Min         Gls Ast G+A G-PK PK PKatt PKm     Tkl TklW   
0    W 4-1     Y  90           0   0   0    0  0     0   0       9    5   
1    W 2-1     Y  90           0   0   0    0  0     0   0       8    7   
2    L 1-3     Y  90           0   0   0    0  0     0   0       3    2   
3    L 0-2     Y  86           0   0   0    0  0     0   0       9    5   
4    W 3-1     Y  90           0   0   0    0  0     0   0       8    6   
..     ...   ...  ..         ...  ..  ..  ... ..   ...  ..     ...  ...   
595  L 0-1     Y  90           0   0   0    0  0     0   0       2    1   
596  W 1-0     Y  90           1   0   1    1  0     0   0       1    1   
597  W 2-1     Y  90           0   0   0    0  0     0   0       1    0   
598  L 1-3     Y  90           0   0   0    0  0     0   0       1    0   
599  L 0-2     Y  90           0   0   0    0  0     0   0       1    1   

                            Challenges                 Blocks         General  \
    Def 3rd Mid 3rd Att 3rd        Tkl Att   Tkl% Lost Blocks Sh Pass     Int   
0         4       4       1          5   6   83.3    1      3  0    3       3   
1         6       2       0          6   7   85.7    1      2  1    1       4   
2         2       1       0          2   2  100.0    0      2  0    2       8   
3         8       0       1          7  10   70.0    3      1  0    1       2   
4         3       5       0          4   6   66.7    2      2  0    2       3   
..      ...     ...     ...        ...  ..    ...  ...    ... ..  ...     ...   
595       0       1       1          1   2   50.0    1      0  0    0       0   
596       1       0       0          1   1  100.0    0      2  0    2       1   
597       1       0       0          1   2   50.0    1      0  0    0       1   
598       0       0       1          0   1    0.0    1      2  1    1       1   
599       0       0       1          1   1  100.0    0      1  0    1       1   

                      

In [87]:
data_combined = pd.concat([data1, data2], axis=0, ignore_index=True)
data_combined = data_combined[~data_combined.duplicated(subset=[('General', 'Player'), ('General', 'Date')], keep='first')]
data_combined

General                                                                 \
          Rk           Player        Date     Age   Nation             Team   
0          1          Murillo  2025-01-14  22-194   br BRA  Nott'ham Forest   
1          2     Dean Huijsen  2025-03-09  19-329   es ESP      Bournemouth   
2          3           Morato  2024-12-29  23-182   br BRA  Nott'ham Forest   
3          4  Maxence Lacroix  2025-02-22  24-322   fr FRA   Crystal Palace   
4          5     Jan Bednarek  2024-12-29  28-261   pl POL      Southampton   
...      ...              ...         ...     ...      ...              ...   
1395     596   Marc Cucurella  2025-03-16  26-237   es ESP          Chelsea   
1396     597   Marc Cucurella  2025-03-09  26-230   es ESP          Chelsea   
1397     598   Marc Cucurella  2025-04-20  26-272   es ESP          Chelsea   
1398     599    Jacob Greaves  2025-05-25  24-255  eng ENG     Ipswich Town   
1399     600    Jacob Greaves  2025-05-18  24-248  eng ENG     Ipswich Town   

                                                                              \
     Unnamed: 8_level_1             Opp                Comp Result Start Min   
0                   NaN       Liverpool  eng Premier League  D 1-1     Y  90   
1                     @       Tottenham  eng Premier League  D 2-2     Y  90   
2                     @         Everton  eng Premier League  W 2-0     Y  90   
3                     @          Fulham  eng Premier League  W 2-0     Y  90   
4                     @  Crystal Palace  eng Premier League  L 1-2     Y  90   
...                 ...             ...                 ...    ...   ...  ..   
1395                  @         Arsenal  eng Premier League  L 0-1     Y  90   
1396                NaN  Leicester City  eng Premier League  W 1-0     Y  90   
1397                  @          Fulham  eng Premier League  W 2-1     Y  90   
1398                NaN        West Ham  eng Premier League  L 1-3     Y  90   
1399                  @  Leicester City  eng Premier League  L 0-2     Y  90   

     Performance                           Tackles                       \
             Gls Ast G+A G-PK PK PKatt PKm     Tkl TklW Def 3rd Mid 3rd   
0              0   0   0    0  0     0   0       4    4       3       1   
1              0   0   0    0  0     0   0       2    1       2       0   
2              0   0   0    0  0     0   0       4    3       4       0   
3              0   0   0    0  0     0   0       4    3       3       1   
4              0   0   0    0  0     0   0       4    2       1       3   
...          ...  ..  ..  ... ..   ...  ..     ...  ...     ...     ...   
1395           0   0   0    0  0     0   0       2    1       0       1   
1396           1   0   1    1  0     0   0       1    1       1       0   
1397           0   0   0    0  0     0   0       1    0       1       0   
1398           0   0   0    0  0     0   0       1    0       0       0   
1399           0   0   0    0  0     0   0       1    1       0       0   

             Challenges                 Blocks         General              \
     Att 3rd        Tkl Att   Tkl% Lost Blocks Sh Pass     Int Tkl+Int Clr   
0          0          1   2   50.0    1      4  1    3       2       6  17   
1          0          2   3   66.7    1      1  1    0       3       5  16   
2          0          2   3   66.7    1      1  0    1       1       5  14   
3          0          2   2  100.0    0      2  1    1       1       5  12   
4          0          1   2   50.0    1      1  1    0       6      10  12   
...      ...        ...  ..    ...  ...    ... ..  ...     ...     ...  ..   
1395       1          1   2   50.0    1      0  0    0       0       2   5   
1396       0          1   1  100.0    0      2  0    2       1       2   7   
1397       0          1   2   50.0    1      0  0    0       1       2   6   
1398       1          0   1    0.0    1      2  1    1       1       2   9   
1399       1          1 

In [88]:
data_combined['CBIT'] = data_combined[('General', 'Tkl+Int')] + data_combined[('Blocks', 'Blocks')] + data_combined[('General', 'Clr')]

In [89]:
df = data_combined[(data_combined.CBIT>=10) & (data_combined[('General', 'Pos.')] == 'DF')]
df

General                                                                 \
          Rk           Player        Date     Age   Nation             Team   
0          1          Murillo  2025-01-14  22-194   br BRA  Nott'ham Forest   
1          2     Dean Huijsen  2025-03-09  19-329   es ESP      Bournemouth   
2          3           Morato  2024-12-29  23-182   br BRA  Nott'ham Forest   
3          4  Maxence Lacroix  2025-02-22  24-322   fr FRA   Crystal Palace   
4          5     Jan Bednarek  2024-12-29  28-261   pl POL      Southampton   
...      ...              ...         ...     ...      ...              ...   
1392     593      Dara O'Shea  2025-04-05  26-032   ie IRL     Ipswich Town   
1393     594      Dara O'Shea  2024-11-02  25-243   ie IRL     Ipswich Town   
1394     595      Dara O'Shea  2024-11-10  25-251   ie IRL     Ipswich Town   
1396     597   Marc Cucurella  2025-03-09  26-230   es ESP          Chelsea   
1398     599    Jacob Greaves  2025-05-25  24-255  eng ENG     Ipswich Town   

                                                                              \
     Unnamed: 8_level_1             Opp                Comp Result Start Min   
0                   NaN       Liverpool  eng Premier League  D 1-1     Y  90   
1                     @       Tottenham  eng Premier League  D 2-2     Y  90   
2                     @         Everton  eng Premier League  W 2-0     Y  90   
3                     @          Fulham  eng Premier League  W 2-0     Y  90   
4                     @  Crystal Palace  eng Premier League  L 1-2     Y  90   
...                 ...             ...                 ...    ...   ...  ..   
1392                NaN          Wolves  eng Premier League  L 1-2     Y  90   
1393                NaN  Leicester City  eng Premier League  D 1-1     Y  90   
1394                  @       Tottenham  eng Premier League  W 2-1     Y  90   
1396                NaN  Leicester City  eng Premier League  W 1-0     Y  90   
1398                NaN        West Ham  eng Premier League  L 1-3     Y  90   

     Performance                           Tackles                       \
             Gls Ast G+A G-PK PK PKatt PKm     Tkl TklW Def 3rd Mid 3rd   
0              0   0   0    0  0     0   0       4    4       3       1   
1              0   0   0    0  0     0   0       2    1       2       0   
2              0   0   0    0  0     0   0       4    3       4       0   
3              0   0   0    0  0     0   0       4    3       3       1   
4              0   0   0    0  0     0   0       4    2       1       3   
...          ...  ..  ..  ... ..   ...  ..     ...  ...     ...     ...   
1392           0   1   1    0  0     0   0       1    0       1       0   
1393           0   0   0    0  0     0   0       0    0       0       0   
1394           0   0   0    0  0     0   0       2    2       0       2   
1396           1   0   1    1  0     0   0       1    1       1       0   
1398           0   0   0    0  0     0   0       1    0       0       0   

             Challenges                 Blocks         General              \
     Att 3rd        Tkl Att   Tkl% Lost Blocks Sh Pass     Int Tkl+Int Clr   
0          0          1   2   50.0    1      4  1    3       2       6  17   
1          0          2   3   66.7    1      1  1    0       3       5  16   
2          0          2   3   66.7    1      1  0    1       1       5  14   
3          0          2   2  100.0    0      2  1    1       1       5  12   
4          0          1   2   50.0    1      1  1    0       6      10  12   
...      ...        ...  ..    ...  ...    ... ..  ...     ...     ...  ..   
1392       0          0   0    NaN    0      2  2    0       1       2  10   
1393       0          0   0    NaN    0      5  5    0       2       2   8   
1394       0          1   2   50.0    1      4  3    1       0       2   9   
1396       0          1   1  100.0    0      2  0    2       1       2   7   
1398       1          0 

In [90]:
len(df[df[('General', 'Player')] == 'Murillo'])

17

In [91]:
len(df[df[('General', 'Player')] == 'James Tarkowski'])

19

In [92]:
len(df[df[('General', 'Player')] == 'Jan Bednarek'])

13

In [93]:
len(df[df[('General', 'Player')] == 'Wout Faes'])

7

In [94]:
len(data_combined[data_combined[('General', 'Player')] == 'Wout Faes'])

8

In [95]:
len(data_combined[data_combined[('General', 'Player')] == 'Murillo'])

19

In [96]:
len(data_combined[data_combined[('General', 'Player')] == 'Murillo'])

19